<a href="https://colab.research.google.com/github/luimui/SS-SSuI/blob/master/02-hidden-markov-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hidden Markov Models - Implementierung
Für eine Smartwatch soll ein System entwickelt werden, das automatisch erkennt, ob der Träger gerade schläft. Die Smartwatch enthält einen Beschleunigungssensor, der entweder den Wert "wenig Bewegung" oder den Wert "viel Bewegung" zurückliefert.    

Zu Anfang ist der Träger mit einer Wahrscheinlichkeit von 50 % wach.   

Wenn der Träger wach ist, ist die Wahrscheinlichkeit, dass er im nächsten Schritt schläft, 20 %.
Wenn der Träger schläft, ist die Wahrscheinlichkeit, dass er im nächsten Zeitschritt wach ist, 30 %.   

Wenn der Träger wach ist, ist die Wahrscheinlichkeit für "viel Bewegung" 60 % und die Wahrscheinlichkeit für "wenig Bewegung" 40 %. Wenn der Träger schläft, ist die Wahrscheinlichkeit für "viel Bewegung" 5 % und die Wahrscheinlichkeit für "wenig Bewegung" 95 %.

## Aufgabe 1
Implementieren Sie zunächst für das oben beschriebene HMM die Modelle für Transition und Observation, wie den Initalzustand des Systems!

In [1]:
#transition modell
T = matrix(c(0.8, 0.2, 0.3, 0.7), nrow=2, ncol=2)

#observation model
O = matrix(c(0.6, 0.4, 0.05, 0.95), nrow=2, ncol=2)


#prior
s0 = c(0.5, 0.5)

In [5]:
O

0.6,0.05
0.4,0.95


## Aufgabe 2
Implementieren Sie eine Funktion `predict(s,T)` die als Parameter eine Prior-Verteilung $p(x_t | y_{1:t})$ als Vektor $s$ und das Transitionsmodell als Matrix `A` bekommt und als Ausgabe die Verteilung $p(x_{t+1} | y_{1:t})$ liefert.

In [40]:
predict = function(s,T){
    predict = (T %*% s)[,1] #Siehe Aufgabe 5 mit [,1]
    predict
}

s = predict(s0,T)
print(s)

[1] 0.55 0.45


## Aufgabe 3
Implementieren Sie eine Funktion `update(s,O,y)` die als Parameter eine Verteilung $p(x_{t+1} | y_{1:t})$ als Vektor $s$, ein Observationsmodell als Matrix `O` und eine Beobachtung $y_{t+1}$ als Integer `y` bekommt und als Ausgabe $p(x_{t+1} | y_{1:t+1})$ liefert.

Sensor liefert "w" für wenig Bewegung, "v" für wenig Bewegung

In [41]:
update = function(s,O,y){
    o_row = ifelse(y == "v", 1, 2)
    si = O[o_row,] * s
    si / sum(si)
}

In [42]:
si = update(s,O,"w")
print(si)

[1] 0.3397683 0.6602317


## Aufgabe 4
Implementieren Sie eine Funktion `filter(s0,Y,T,O)` die als Parameter eine Prior-Verteilung $p(x_0)$ als Vektor `s0`, ein Transitions- und Beobachtungsmodell als Matrizen `T` und `O` und eine Sequenz von Beobachtungen $y_{1:T}$ als Vektor `yy` bekommt und für jedes $t = 1,...,T$ die Verteilung $p(x_t | y_{1:t})$ als Vektor berechnet und als Aufgabe somit eine $T \times 2$-Matrix liefert.

In [43]:
filter = function(s0,O,T,yy){

    s = s0
    ss = list(s)

    for(y in yy){
      si = predict(s,T)
      s  = update(si,O,y)
      ss = c(ss,list(s))
    }

    ss

}



In [44]:
list_of_lists = (filter(s0,O,T,c("w","w","w")))
filtermatrix <- do.call("cbind",list_of_lists)
print(filtermatrix)

     [,1]      [,2]      [,3]     [,4]
[1,]  0.5 0.3397683 0.2717807 0.245482
[2,]  0.5 0.6602317 0.7282193 0.754518


## Aufgabe 5
Testen Sie nun ihre Implementierung mit der Funktion `testHMM`.

In [45]:
testHMM = function(){
    #observation sequence
    yy = c("W","W","V")

    ss = filter(s0,O,T,yy)
    ss
}

ss = testHMM()
ss

[[1]]
[1] 0.5 0.5

[[2]]
[1] 0.3397683 0.6602317

[[3]]
[1] 0.2717807 0.7282193

[[4]]
[1] 0.245482 0.754518